In [1]:
from dataclasses import dataclass
from pyrs.pyrs import passes, to_hugr_dot, to_hugr, check_soundness
from pytket.circuit import Circuit, OpType
from pathlib import Path
from pytket.qasm.qasm import circuit_from_qasm
from pytket.passes.auto_rebase import auto_rebase_pass
from pytket.passes import FlattenRelabelRegistersPass, RemoveRedundancies
from pytket.utils.stats import gate_counts
from pytket.utils.results import compare_unitaries
import graphviz


In [2]:
direc = Path.home() / "tket/tket/pytket/tests/qasm_test_files/"

files = list(direc.iterdir())

In [7]:

def view_dot(c: Circuit):
    # print(to_hugr_dot(c))
    display(graphviz.Source(to_hugr_dot(c)))


def compare(c: Circuit, c2: Circuit) -> bool:
    from pytket.extensions.qiskit import AerUnitaryBackend
    b = AerUnitaryBackend()
    (u, u2) = (b.run_circuit(circ).get_unitary() for circ in (c, c2))

    res =  compare_unitaries(u, u2)
    # if not res:
    #     print(u)
    #     print(u2)

    return res

In [14]:
def test_file(file: Path):
    try:
        c = circuit_from_qasm(file)
    except Exception as e:
        print(e)
        return
    try:
        auto_rebase_pass({OpType.PhasedX, OpType.ZZPhase, OpType.Rz}).apply(c)
    except RuntimeError as e:
        if "CustomGate" in str(e):
            return
        else:
            raise e

    # tket2 expects simple register names for now
    FlattenRelabelRegistersPass().apply(c)
    RemoveRedundancies().apply(c)
    print(f"{file.name}")
    c2, moves = passes.greedy_depth_reduce(c)
    assert (gc:=gate_counts(c)) == gate_counts(c2)
    print(f"{moves} {c.depth()} {c2.depth()}")
    if gc.get(OpType.Measure, 0) or c.free_symbols():
        return

    if 0<c.n_qubits < 9 and c.n_gates < 1000:
        if not compare(c, c2):
            print("unitary fail")
            for circ in (c, c2):
                print("\n".join(map(str,circ.get_commands())))
                print("\n")

In [15]:
for f in files:
    test_file(f)


test13.qasm
22 23 21
test1.qasm
8 64 62
Cannot include extern calls without a wasm module specified.
File:/Users/seyon/tket/tket/pytket/tests/qasm_test_files/test18.qasm: 
Line:26. 
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
test14.qasm
60 228 216
test15.qasm
3 24 23
test7.qasm
2 8 7
test19.qasm
133 529 507
zzphase.qasm
0 1 1
test8.qasm
403 1745 1621
test16.qasm
0 0 0
Cannot parse gate of type: gatedoesntexist
File:/Users/seyon/tket/tket/pytket/tests/qasm_test_files/test4.qasm: 
Line:6. 
test6_output.qasm
1 9 8
Cannot include extern calls without a wasm module specified.
File:/Users/seyon/tket/tket/pytket/tests/qasm_test_files/test18_output.qasm: 
Line:19. 
test5.qasm
0 2 2
Cannot include extern calls without a wasm module specified.
File:/Users/seyon/tket/tket/pytket/tests/qasm_test_files/test17.qasm: 
Line:24. 
test9.qasm
2 15 15
test10.qasm
1 10 10
Circuit does not contain unit with id: q[4]
'utf-8' codec can't decode byte 0xff in position 0: invalid star